<a href="https://colab.research.google.com/github/HakanOznam/Makine-Ogrenmesi-ile-Satranc-Oyun-Sonu-Tahmini/blob/main/Chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kütüphanelerin import edilmesi ve veri kümesinin dahil edilmesi

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay ,classification_report

data=pd.read_excel("games.xlsx")

Analizin doğruluğunu arttırmak için hamle sayısı 10'dan küçük olan maçları veri kümesinden silme işlemi

In [2]:
data=data.drop(data.loc[data['turns']<=10].index)
data=data.reset_index(drop=True)

Eksik veri kontrolü

In [3]:
data.isnull().sum()

id                   0
rated                0
game_time         9000
turns                0
victory_status       0
winner               0
white_id             0
white_rating         0
black_id             0
black_rating         0
moves                0
opening_eco          0
opening_name         0
dtype: int64

Eksik verilerin IterativeImputer metodu ile işlenmesi 

In [4]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
testmissing = data.iloc[:9000,2:4]
trainmissing = data.iloc[9000:,2:4]
imp = IterativeImputer(random_state=0)
imp.fit(trainmissing)
data.iloc[:9000,2:4] = np.round(imp.transform(testmissing))
data.isnull().sum()

id                0
rated             0
game_time         0
turns             0
victory_status    0
winner            0
white_id          0
white_rating      0
black_id          0
black_rating      0
moves             0
opening_eco       0
opening_name      0
dtype: int64

Kategorik verilerin sayısal verilere dönüştürme işlemi

In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

#rated özniteliği için 
data.iloc[:,1] = le.fit_transform(data.iloc[:,1])

#victory status özniteliği için
ohe = preprocessing.OneHotEncoder()
vicstatus = data.iloc[:,4].to_numpy()
vicstatus = vicstatus.reshape(-1,1)
ohe_vicstatus = ohe.fit_transform(vicstatus).toarray()
dfvicstatus=pd.DataFrame(data=ohe_vicstatus,columns=["draw","mate","outoftime","resign"])

Oyun açılış(opening_eco) özniteliğinin sayısal verilere dönüştürme işlemi

In [ ]:
eco = data['opening_eco']
for i in range(len(eco)):  
    eco[i]=eco[i].replace("A","1")
    eco[i]=eco[i].replace("B","2")
    eco[i]=eco[i].replace("C","3")
    eco[i]=eco[i].replace("D","4")
    eco[i]=eco[i].replace("E","5")

eco=pd.to_numeric(eco)

Verilerin birleştirilmesi, etiket veri kolonunun belirlenmesi

In [7]:
X = pd.concat([data["rated"],data["game_time"],data["turns"],dfvicstatus,data["black_rating"],data["white_rating"]],axis=1)

y = data["winner"]
y = le.fit_transform(y)
classes =["black","draw","white"]
X.head()

,rated,game_time,turns,draw,mate,outoftime,resign,black_rating,white_rating
0,0,365889.0,13.0,0.0,0.0,1.0,0.0,1191,1500
1,1,396358.0,16.0,0.0,0.0,0.0,1.0,1261,1322
2,1,853391.0,61.0,0.0,1.0,0.0,0.0,1500,1496
3,1,853391.0,61.0,0.0,1.0,0.0,0.0,1454,1439
4,1,1198705.0,95.0,0.0,1.0,0.0,0.0,1469,1523


Verilerin standardize edilmesi


In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_sc = sc.fit_transform(X)


KFold algoritması kullanılarak modellerin sonuçlarını yazdıran fonksiyon tanımlanması

In [10]:
def output(est,X,color):
    for model in est:
        accu = cross_val_score(estimator = model, X = X, y=y,cv=5)
        y_pred = cross_val_predict(model,X,y,cv=5)
        conf_matrix = confusion_matrix(y, y_pred)        
        """görselleştirme"""
        #disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix,display_labels=["black","draw","white"])
        #disp.plot(cmap=color)
        #disp.ax_.set_title(model)
        print("-------- ",model," report -------- \n")
        print(classification_report(y,y_pred,target_names=classes))
        print("\n")

Modellerin import edilmesi

In [15]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(random_state=0)

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()


GridSearchCV algoritması ile Random Forest Classifier

In [12]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

parameters = [{'n_estimators': [50,100,150,200], 'criterion':['gini',"entropy"]}]
grids = GridSearchCV(estimator = rfc, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)
grids.fit(X_sc, y)
best_accuracy , best_parameter = grids.best_score_ , grids.best_params_

In [13]:
best_parameter

{'criterion': 'gini', 'n_estimators': 200}

In [22]:
grid_rfc = RandomForestClassifier(n_estimators=200,criterion="gini")

GridSearchCV algoritması ile KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
"""
parameters = [{'n_neighbors': [3,5,7,9,11,13,15,17,19,21], 'weights':['uniform',"distance"],"metric" : ["euclidean","manhattan","minkowski"]}]
grids = GridSearchCV(estimator = knn, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)
grids.fit(X_sc, y)
best_accuracy , best_parameter = grids.best_score_ , grids.best_params_
"""

In [19]:
grid_knn = KNeighborsClassifier(n_neighbors=21,weights="distance",metric="manhattan")

GridSearchCV algoritması ile Support Vector Classifier

In [ ]:
from sklearn.svm import SVC
svm = SVC()
"""
parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel':['linear']}, {'C': [0.25, 0.5, 0.75, 1], 'kernel':['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5]}]
grids = GridSearchCV(estimator = svm, param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)
grids.fit(X_sc, y)
best_accuracy , best_parameter = grids.best_score_ , grids.best_params_
"""

In [20]:
grid_svm = SVC(C=0.75,kernel="linear")

Parametre kullanılmayan modellerin sonuçları

In [18]:
est = [logreg,clf,rfc,knn,gnb]
output(est,X_sc,"pink")

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


--------  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)  report -------- 

              precision    recall  f1-score   support

       black       0.65      0.58      0.61      8831
        draw       0.99      0.95      0.97       910
       white       0.65      0.71      0.68      9636

    accuracy                           0.66     19377
   macro avg       0.76      0.75      0.75     19377
weighted avg       0.66      0.66      0.66     19377



--------  DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min

GridSearchCV uygulanan üç modelin sonuçları

In [23]:
grid_est = [grid_rfc,grid_knn,grid_svm]
output(grid_est,X_sc,"hot")


--------  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)  report -------- 

              precision    recall  f1-score   support

       black       0.67      0.66      0.67      8831
        draw       1.00      0.95      0.97       910
       white       0.69      0.71      0.70      9636

    accuracy                           0.70     19377
   macro avg       0.79      0.77      0.78     19377
weighted avg       0.70      0.70      0.70     19377



--------  KNeighborsClassifier(algorithm=

Boyut azaltma tekniği olan LDA algoritması 

In [24]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 2)
X_lda = lda.fit_transform(X_sc, y)

LDA ve GridSearchCV uygulanan modellerin sonuçları

In [25]:
grid_lda_est = [logreg,clf,grid_rfc,grid_knn,gnb]
output(grid_lda_est,X_lda,"bone")

--------  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)  report -------- 

              precision    recall  f1-score   support

       black       0.65      0.58      0.61      8831
        draw       1.00      0.95      0.97       910
       white       0.65      0.71      0.68      9636

    accuracy                           0.66     19377
   macro avg       0.76      0.75      0.76     19377
weighted avg       0.66      0.66      0.66     19377



--------  DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min